In [7]:
import psycopg2
import time
import pytz
import threading
import os
import datetime
import pandas as pd
from finvizfinance.quote import finvizfinance
from concurrent.futures import ThreadPoolExecutor, as_completed

In [8]:
def get_connection():
    try:
        connection = psycopg2.connect(
            dbname=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            host=os.getenv('DB_HOST'),
            options='-c statement_timeout=1020000'
        )

    except psycopg2.OperationalError as e:
        print(f'Error: {e}')
        return None

    else:
        print('Connected Established!')
        return connection

In [20]:
conn = get_connection()
cursor = conn.cursor()

Connected Established!


In [ ]:
query = '''
SELECT Ticker FROM Stocks
'''

cursor.execute(query)
tickers = cursor.fetchall()
tickers = [ticker[0] for ticker in tickers]

In [ ]:
news_dfs = {}

def get_news(ticker):
    print(f'Fetching headlines for {ticker}...')
    stock = finvizfinance(ticker)
    news_df = stock.ticker_news()
    return news_df 

def fetch_headlines(ticker):
    try:
        news_dfs[ticker] = get_news(ticker)
    except Exception as e:
        print(f"Error fetching headlines for {ticker}: {e}")
        fetch_headlines(ticker)

with ThreadPoolExecutor(max_workers=2) as executor:
    for i, ticker in enumerate(tickers):
        executor.submit(fetch_headlines, ticker)

%store news_dfs

In [ ]:
current_time = datetime.datetime.now(pytz.timezone('US/Eastern'))
et_now = current_time.replace(second=0, microsecond=0).replace(tzinfo=None)

In [ ]:
def convert_to_ET(dt):
    eastern = pytz.timezone('US/Eastern')
    return dt.astimezone(eastern)

def extract_headlines_within_one_day(df, current_time):
    one_day_ago = current_time - pd.Timedelta(days=1)
    return df[df['Date'] >= one_day_ago]

def extract_headlines_within_one_week(df, current_time):
    one_week_ago = current_time - pd.Timedelta(weeks=1)
    return df[df['Date'] >= one_week_ago]

def extract_headlines_within_one_month(df, current_time):
    one_month_ago = current_time - pd.Timedelta(days=30)
    return df[df['Date'] >= one_month_ago]

def extract_headlines_within_timeframes(dictionary_of_dataframes):
    current_time_ET = convert_to_ET(datetime.datetime.now())
    current_time_ET = current_time_ET.replace(second=0, microsecond=0).replace(tzinfo=None)

    headlines_within_timeframes = {}

    for stock, df in dictionary_of_dataframes.items():
        daily_headlines = extract_headlines_within_one_day(df, current_time_ET)
        weekly_headlines = extract_headlines_within_one_week(df, current_time_ET)
        monthly_headlines = extract_headlines_within_one_month(df, current_time_ET)
        
        headlines_within_timeframes[stock] = {
            'Daily': daily_headlines,
            'Weekly': weekly_headlines,
            'Monthly': monthly_headlines
        }
    
    return headlines_within_timeframes

df = extract_headlines_within_timeframes(news_dfs)

%store df

In [9]:
%store -r df news_dfs

In [10]:
for ticker, period_data in df.items():
    for period, dataframe in period_data.items():

        if period == 'Daily':
            dataframe['is_daily'] = True
            dataframe['is_weekly'] = True
            dataframe['is_monthly'] = True

        elif period == 'Weekly':
            dataframe['is_daily'] = False
            dataframe['is_weekly'] = True
            dataframe['is_monthly'] = True

        elif period == 'Monthly':
            dataframe['is_daily'] = False
            dataframe['is_weekly'] = False
            dataframe['is_monthly'] = True

        desired_columns_order = ['Date', 'Title', 'Link','is_daily', 'is_weekly', 'is_monthly']
        dataframe = dataframe[desired_columns_order]

In [11]:
import pandas as pd

dfs = []

for ticker, data in df.items():
    dataframe['Ticker'] = ticker
    dataframe['Period'] = period

    dfs.append(dataframe)

merged_df = pd.concat(dfs, ignore_index=True)

filtered_df = merged_df.drop_duplicates(subset=['Ticker', 'Title'])

In [12]:
filtered_df['Date'] = filtered_df['Date'].astype(str)

filtered_df['is_daily'] = filtered_df['is_daily'].astype(str).str.lower()
filtered_df['is_weekly'] = filtered_df['is_weekly'].astype(str).str.lower()
filtered_df['is_monthly'] = filtered_df['is_monthly'].astype(str).str.lower()

values = [tuple(row) for row in filtered_df[['Date', 'Ticker', 'Title', 'is_daily', 'is_weekly', 'is_monthly']].values]

/tmp/ipykernel_17759/4103925058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date'] = filtered_df['Date'].astype(str)
/tmp/ipykernel_17759/4103925058.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['is_daily'] = filtered_df['is_daily'].astype(str).str.lower()
/tmp/ipykernel_17759/4103925058.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [15]:
def insert_rows(values):
    conn = get_connection()
    cur = conn.cursor()

    cur.execute('DELETE FROM headlines')
    conn.commit()

    query = '''
    INSERT INTO headlines (publication_timestamp, ticker, headline, is_daily, is_weekly, is_monthly)
    VALUES (%s, %s, %s, %s, %s, %s)
    '''

    for value in values:
        cur.execute(query, value)
        conn.commit()
        print(value)

    cur.close()
    conn.close()

num_threads = 16  
chunk_size = len(values) // num_threads + 1
value_chunks = [values[i:i+chunk_size] for i in range(0, len(values), chunk_size)]

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(insert_rows, chunk) for chunk in value_chunks]
    for future in futures:
        future.result()


Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
Connected Established!
('2024-02-13 16:45:00', 'MAR', 'UPDATE 1-The new normal: travel companies temper expectations for 2024', 'false', 'true', 'true')
('2024-02-14 16:59:00', 'AWK', 'American Water Works Co Inc (AWK) Reports Solid 2023 Earnings and Raises 2024 EPS Guidance', 'false', 'true', 'true')
('2024-02-15 14:36:00', 'CMG', 'Giving Uber Its Due', 'false', 'true', 'true')
('2024-02-07 09:14:00', 'MRNA', 'Moderna Chief Executive Officer Stéphane Bancel Elected to National Academy of Engineering', 'false', 'false', 'true')
('2024-02-05 15:33:00', 'MSCI', 'EMERGING MARKETS-Latam currencies edge lower as traders pare Fed rate cut bets, Argentin